# 네이버 쇼핑
* Scroll down 할 때 정보가 업데이트 된다. 


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import time

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [72]:
# Functions 
def scr_down():
    old_height = driver.execute_script('return window.scrollY')
    while True:    
        # 한 번 스크롤 다운 
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)

        # 스크롤 길이 탐색
        new_height = driver.execute_script('return window.scrollY')
        print(new_height, end=' > ')
        
        if new_height == old_height:
            break
        old_height = new_height
        
    print('End!')
    
def get_info(item):
    title = item.find_element(By.CLASS_NAME, 'basicList_title__VfX3c').text

    price = item.find_element(By.CLASS_NAME, 'price_num__S2p_v').text

    specs = item.find_elements(By.CLASS_NAME, 'basicList_detail__8kFi2')
    spec_lst = [spec.text for spec in specs if spec.text != '']

    types = item.find_elements(By.CLASS_NAME, 'basicList_category__cXUaZ')
    type_lst = [type.text for type in types]
    type = type_lst[-1]

    etc = item.find_elements(By.CLASS_NAME, 'basicList_etc__LSkN_')
    etc_lst = [e.text for e in etc[:-1]]
    
    return [title, price, spec_lst, type, etc_lst]
    

In [67]:
# Chrome 열기
driver.get('https://shopping.naver.com/')

In [68]:
# 검색어 입력 
keyword = '냉장고'
pg_no = 5

input_box = driver.find_element(By.CSS_SELECTOR, 'input._searchInput_search_text_3CUDs')

input_box.send_keys(keyword)            # 입력
input_box.send_keys(Keys.ENTER)         # Enter
# input_box.clear()                     # 입력된 내용 지우기

In [73]:
# items 모두 읽기
scr_down()
css_sel = '#__next > div > div.style_container__UxP6u > div.style_inner__i4gKy > div.style_content_wrap__Cdqnl > div.style_content__xWg5l > ul > div > div > li'
items = driver.find_elements(By.CSS_SELECTOR, css_sel)
len('한 페이지 당 item 수:',items)

1556 > 3620 > 5630.66650390625 > 7698.66650390625 > 9718.6669921875 > 9914.6669921875 > 9914.6669921875 > End!


46

In [ ]:
item_lst = []
for pg in range(1,pg_no+1):
    if pg != 1:
        pg_box = driver.find_element(By.CLASS_NAME, 'pagination_num__b1BF2')
        pg_box.find_element(By.LINK_TEXT, '2').click()
        print(f'{pg}', end=' > ')
    
    print()
    
    time.sleep(1)
    
    # items 모두 읽기
    scr_down()
    css_sel = '#__next > div > div.style_container__UxP6u > div.style_inner__i4gKy > div.style_content_wrap__Cdqnl > div.style_content__xWg5l > ul > div > div > li'
    items = driver.find_elements(By.CSS_SELECTOR, css_sel)
    len('한 페이지 당 item 수:',items)
    
    # item 읽기 
    for item in items:
        item_lst.append(get_info(item))
    
print('작업이 왼료되었습니다.')

In [299]:
# Total Code
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import time
import re 

# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Functions 
def scr_down():
    old_height = driver.execute_script('return window.scrollY')
    while True:    
        # 한 번 스크롤 다운 
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)

        time.sleep(1)
        
        # 스크롤 길이 탐색
        new_height = driver.execute_script('return window.scrollY')
        print(new_height, end=' > ')
        
        if new_height == old_height:
            break
        old_height = new_height
        
    print('End!')

def etc_func(etc):
    keys = [re.sub('[^가-힣]','',i) for i in etc]
    values = [re.sub('[^0-9.]','',i) for i in etc]
    
    etc_dict =  dict(zip(keys, values))
    
    # 필요한 key 생성
    for a in ['리뷰', '별점', '등록일', '찜하기']:
        if a not in etc_dict:
            etc_dict[a] = ''
            break

    # 필요없는 key 삭제
    for k in etc_dict.keys():
        if k not in ['리뷰', '별점', '등록일', '찜하기']:
            del etc_dict[k]
            break 
    
    etc_dict = dict(sorted(etc_dict.items()))
    
    return list(etc_dict.values())

def get_info(item):
    title = item.find_element(By.CLASS_NAME, 'basicList_title__VfX3c').text

    price = item.find_element(By.CLASS_NAME, 'price_num__S2p_v').text
    price = re.sub('[^0-9]','',price)
    
    specs = item.find_elements(By.CLASS_NAME, 'basicList_detail__8kFi2')
    spec_lst = [spec.text for spec in specs if spec.text != '']

    types = item.find_elements(By.CLASS_NAME, 'basicList_category__cXUaZ')
    type_lst = [type.text for type in types]
    item_type = type_lst[-1]

    etc = item.find_elements(By.CLASS_NAME, 'basicList_etc__LSkN_')
    review = '리뷰' + item.find_element(By.CLASS_NAME, 'basicList_num__sfz3h').text
    if '별점' in etc[0].text:
        star = item.find_element(By.CLASS_NAME, 'basicList_star__UzKiv').text
    else:
        star = '별점'
    etc_temp = [review, star]
    etc_temp.extend([e.text for e in etc[:-1]])
    etc_lst = etc_func(etc_temp)

    info_lst = [title, price, spec_lst, item_type]
    info_lst.extend(etc_lst)
    
    return info_lst

#----------------------------------Code Start----------------------------------#
print('작업을 시작합니다.')

# Chrome 열기
driver.get('https://shopping.naver.com/')

time.sleep(0.2)

# 검색어 입력 
keyword = '냉장고'
pg_no = 3

input_box = driver.find_element(By.CSS_SELECTOR, 'input._searchInput_search_text_3CUDs')

input_box.send_keys(keyword)            # 입력
input_box.send_keys(Keys.ENTER)         # Enter
# input_box.clear()                     # 입력된 내용 지우기

time.sleep(1)

# Items 읽어오기
item_lst = []
for pg in range(1,pg_no+1):
    if pg != 1:
        pg_box = driver.find_element(By.CLASS_NAME, 'pagination_num__b1BF2')
        pg_box.find_element(By.LINK_TEXT, f'{pg}').click()
    
    time.sleep(0.5)
    
    # items 모두 읽기
    scr_down()
    css_sel = '#__next > div > div.style_container__UxP6u > div.style_inner__i4gKy > div.style_content_wrap__Cdqnl > div.style_content__xWg5l > ul > div > div > li'
    items = driver.find_elements(By.CSS_SELECTOR, css_sel)
    print('한 페이지 당 item 수:',len(items))
    
    # item 읽기 
    for item in items:
        item_lst.append(get_info(item))
    print(f'-----{pg} 페이지를 모두 읽었습니다-----')
    
print('작업이 왼료되었습니다.')

작업을 시작합니다.
10325 > 10325 > End!
한 페이지 당 item 수: 46
-----1 페이지를 모두 읽었습니다-----
10178 > 10178 > End!
한 페이지 당 item 수: 46
-----2 페이지를 모두 읽었습니다-----
10129 > 10129 > End!
한 페이지 당 item 수: 46
-----3 페이지를 모두 읽었습니다-----
작업이 왼료되었습니다.


In [300]:
df = pd.DataFrame(item_lst)
df.columns = ['상품명', '가격(최저)', '스펙', '유형', '등록일', '리뷰', '별점','찜하기'] 
# df.head(30)

In [301]:
# 데이터 타입 변경 
import numpy as np
df = df.replace('',np.NaN)

df[['가격(최저)','리뷰','찜하기']] = df[['가격(최저)','리뷰','찜하기']].astype(int)
df['별점'] = df['별점'].astype(float)

df.dtypes

상품명        object
가격(최저)      int32
스펙         object
유형         object
등록일        object
리뷰          int32
별점        float64
찜하기         int32
dtype: object

In [302]:
df.sort_values(by='등록일', ascending=False)

,상품명,가격(최저),스펙,유형,등록일,리뷰,별점,찜하기
120,렌탈냉장고 렌탈 삼성 엘지 위니아 비스포크 디오스 3/5년약정,14900,"[에너지효율 : 3등급, 도어개수 : 2도어, 내부조명 : 트리플LED, 냉각 : ...",일반형냉장고,2022.10.,1020,NaN,1
66,렌탈LG 얼음정수기냉장고렌탈 양문형 디오스 오브제 3-7년의무 디디하이브,40900,[],양문형냉장고,2022.09.,409,NaN,409
74,렌탈냉장고 렌탈 삼성 엘지 위니아 비스포크 디오스 3/5년약정,14900,"[에너지효율 : 1등급, 도어개수 : 2도어, 내부조명 : 트리플LED, 냉각 : ...",일반형냉장고,2022.09.,810,NaN,4
117,캐리어 KRNC250ESM1,452950,"[에너지효율 : 3등급, 용량 : 250L, 크기 : 540 x 1,800 x 59...",일반형냉장고,2022.08.,36,NaN,36
6,삼성전자 비스포크 RF85B90P1AP,1779600,"[에너지효율 : 1등급, 용량 : 875L, 크기 : 912 x 1,853 x 93...",양문형냉장고,2022.08.,1293,4.9,287
...,...,...,...,...,...,...,...,...
108,LG전자 B057W,134420,"[에너지효율 : 1등급, 용량 : 46L, 크기 : 44.3 x 50.1 x 45c...",일반형냉장고,2018.04.,295,4.6,406
83,삼성전자 삼성 RT38K5039SL,420890,"[에너지효율 : 4등급, 용량 : 381L, 크기 : 67.5 x 178.5 x 6...",일반형냉장고,2017.11.,516,4.8,624
11,삼성전자 삼성 RS82M6000S8,789990,"[에너지효율 : 2등급, 용량 : 815L, 크기 : 91.2x177.9x90.7c...",양문형냉장고,2017.08.,1088,4.9,1861
45,삼성전자 삼성 RT53K6035SL,592170,"[에너지효율 : 2등급, 용량 : 525L, 크기 : 79 x 185.5 x 72c...",일반형냉장고,2016.03.,741,4.8,609


# 구글 이미지 저장

In [ ]:
print('작업을 시작합니다.')
# 저장소 만들기 
file_path = 'Goolge_imgs/'
if not os.path.isdir(file_path):
    os.mkdir(file_path)

# Chrome 열기
driver.get('https://images.google.com/')

time.sleep(0.2)

# 검색어 입력 
keyword = 'happy expression'
more_no = 0
css_sel1 = 'body > div.L3eUgb > div.o3j99.ikrT4e.om7nvf > form > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.a4bIc > input'
input_box = driver.find_element(By.CSS_SELECTOR, css_sel1)

input_box.send_keys(keyword)            # 입력
input_box.send_keys(Keys.ENTER)         # Enter
# input_box.clear()                     # 입력된 내용 지우기

# 스크롤 다운
for more in range(more_no+1):
    if more != 0:
        button = '#islmp > div > div > div > div.gBPM8 > div.qvfT1 > div.YstHxe > input'
        driver.find_element(By.CSS_SELECTOR, button).click()
        print('더보기 Click!')
        
    time.sleep(1)
        
    scr_down()

driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.HOME)


# 스크린샷
css_sel2 = '#islrg > div.islrc > div > a.wXeWr.islib.nfEiy > div.bRMDJf.islir > img'
images = driver.find_elements(By.CSS_SELECTOR, css_sel2)

for i, image in enumerate(images):
    if i == 50:
        break
    
    time.sleep(0.5)
    
    image.screenshot(file_path + '/' + keyword + '%03d'%(i+1) + '.png')
    
    if (i+1) % 10 == 0:
        print(f'전체 {len(images)}개 중 {i+1}개 저장')

print('작업이 완료되었습니다.')

작업을 시작합니다.
2943.333251953125 > 6577.33349609375 > 13603.3330078125 > 20867.333984375 > 24658 > 24658 > Scroll Down End!
전체 353개 중 10개 저장
전체 353개 중 20개 저장
전체 353개 중 30개 저장
전체 353개 중 40개 저장
전체 353개 중 50개 저장
작업이 완료되었습니다.


# 인스타그램 게시글 긁어오기

In [ ]:
####################################################################################
# 인스타그램 메시지 스크래핑
####################################################################################
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
import requests
import re

# 브라우저 띄우기
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


## 로그인 과정

In [ ]:
# Chrome 열기
driver.get('https://www.instagram.com/explore')

In [ ]:
id_box = driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(1) > div > label > input')
pw_box = driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(2) > div > label > input')

In [ ]:
# login
id = '***************'
pw = '***************'

id_box.send_keys(id)            
pw_box.send_keys(pw)     

In [ ]:
id_box.clear()

In [ ]:

driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(3) > button').click()    

In [ ]:
css_sel = '#mount_0_0_sd > div > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div > div.x78zum5.xdt5ytf.x10cihs4.x1t2pt76.x1n2onr6.x1ja2u2z > section > main > div > div > div > div > button'
driver.find_element(By.CSS_SELECTOR, css_sel).click()

## 검색 과정

In [ ]:
### 검색어 입력받기
while True:
    try:
        keyword = input('> 검색어를 입력하세요! : ')
        N = int(input('> 다운받을 게시물의 수를 입력하세요 : '))
        if keyword == '':
            raise
        if N < 1:
            raise
    except:
        print("@@@ 잘못 입력하셨습니다.")
        continue
    else:
        break
    
print('-'*10, keyword,'검색을 시작합니다','-'*10)


In [ ]:
# Chrome 열기
driver.get('https://www.instagram.com/explore')

In [ ]:
keyword = 'multicampus.official'
article_no = 10
save_img = 1        # 1: 저장 2: 미저장

In [ ]:
input_box = driver.find_element(By.CSS_SELECTOR, 'input._aauy')
input_box.clear()
input_box.send_keys(keyword)
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, 'div._abm4').click()

time.sleep(5)

# 첫번째 게시글 클릭 
driver.find_element(By.CSS_SELECTOR, 'div._aabd._aa8k._aanf').click()



In [ ]:
from datetime import datetime

# 작성자, 게시글
writer = driver.find_element(By.CSS_SELECTOR, 'span._aap6._aap7._aap8').text
article = driver.find_element(By.CSS_SELECTOR, 'span._aacl._aaco._aacu._aacx._aad7._aade').text

# 좋아요
likes = driver.find_element(By.CSS_SELECTOR, 'a > div._aacl._aaco._aacw._aacx._aada._aade').text
likes = re.sub('[^0-9.]','',likes)

# 날짜
post_date = driver.find_element(By.CSS_SELECTOR,  'time._aaqe').get_attribute('datetime')
post_date = post_date.split('.')[0].replace('T','')
post_date = datetime.strptime(post_date,'%Y-%m-%d%H:%M:%S')
# post_date = driver.find_element(By.CSS_SELECTOR,  'time._aaqe').text
# post_date = datetime.strptime(post_date, '%m월 %d, %Y')

# 이미지 url 
img_url = driver.find_element(By.CSS_SELECTOR, 'div._aagu._aato > div._aagv > img').get_attribute('src')


[writer, article, likes, post_date, img_url]



['multicampus.official',
 "올해 여러분과 함께라서 행복했습니다 😊🎉\n⠀\n가족들과 따뜻한 연말 보내시고\n2022년 '🐯검은호랑이의 해' 임인년에는 행복과 기쁨만 가득하세요!\n⠀\n👊커리어 Level UP 치트키👊\n멀티캠퍼스와 기업이 원하는 인재되기\n☞ https://bit.ly/3prx3jy\n\n#2022년 #임인년 #멀티캠퍼스",
 '11',
 datetime.datetime(2021, 12, 27, 1, 23, 48),
 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/270003570_286651720182055_8713584600566643597_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=107&_nc_ohc=EQ-OhDR2BBoAX_iWhst&tn=cZSTlXgdkX4kW9-F&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MjczNzYwNzE3ODc1OTYxMjY0MQ%3D%3D.2-ccb7-5&oh=00_AT_rcAd6HcgynvAoCJnD30t0BfGd-v36BsdbpuvGW-jx5w&oe=6352F949&_nc_sid=30a2ef']

In [ ]:
post = 1

In [ ]:
img_url = driver.find_element(By.CSS_SELECTOR, 'div._aagu._aato > div._aagv > img').get_attribute('src')
img_reponse = requests.get(img_url)
filename = 'article' + str(post) + '_img' + str(i+1) + '.png'

with open(file_path + filename, 'wb') as f:
    f.write(img_reponse.content)

i += 1
print(f'{i}', end=' > ')


driver.find_element(By.CSS_SELECTOR, 'button._afxw').click()

0 > 

In [ ]:
# 저장소 만들기 
import os
file_path = 'Instagram_imgs/'
if not os.path.isdir(file_path):
    os.mkdir(file_path)
    
i = 1
while True:
    try:
        if i == 1:
            img_url = driver.find_elements(By.CSS_SELECTOR, 'div._aagu._aato > div._aagv > img')[0].get_attribute('src')
        else:
            img_url = driver.find_elements(By.CSS_SELECTOR, 'div._aagu._aato > div._aagv > img')[1].get_attribute('src')
        img_reponse = requests.get(img_url)
        
        filename = 'article' + str(post) + '_img' + str(i) + '.png'
        with open(file_path + filename, 'wb') as f:
            f.write(img_reponse.content)
        
        print(f'{i}', end=' > ')
        i += 1
        
        driver.find_element(By.CSS_SELECTOR, 'button._afxw').click()
        time.sleep(0.5)

    except: 
        print('> 저장 완료!')
        break 

1 > 2 > 3 > 4 > > 저장 완료!


In [ ]:
test = driver.find_elements(By.CSS_SELECTOR, 'div._aagu._aato > div._aagv > img')
test
# [t.get_attribute('src') for t in test]

[<selenium.webdriver.remote.webelement.WebElement (session="3913a420161f9eab57378e10c8400370", element="aae12051-344e-41fc-ad6c-de127b241c08")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3913a420161f9eab57378e10c8400370", element="d5766046-26ca-491e-9169-95c8cd0ffc5f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3913a420161f9eab57378e10c8400370", element="1878b29c-2889-42e0-a53f-0757edbbea3e")>]

In [ ]:
# 다음 페이지 넘기기 
driver.find_element(By.CLASS_NAME, "_aaqg._aaqh").click()

In [ ]:
# Total Code
####################################################################################
# 인스타그램 메시지 스크래핑
####################################################################################
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from datetime import datetime
import time
import requests
import re
import os 
#--------------------------Chrome 환경 조성--------------------------#
# 브라우저 띄우기
# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Chrome 열기
driver.get('https://www.instagram.com/explore')
time.sleep(5)
#------------------------키워드 및 페이지 설정------------------------#
# 사용자 입력 (input함수로 바꾸기, 일단 테스트)
keyword = 'multicampus.official'
post_no = 3

# 저장소 만들기 
import os
file_path = 'Instagram_imgs/'
if not os.path.isdir(file_path):
    os.mkdir(file_path)
#-----------------------------크롤링 시작-----------------------------#
start = '크롤링 작업을 시작합니다'
print('='*11,start.center(15),'='*11)

# 검색어 입력
input_box = driver.find_element(By.CSS_SELECTOR, 'input._aauy')
input_box.clear()
input_box.send_keys(keyword)
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, 'div._abm4').click()

# 첫번째 게시글 클릭 
time.sleep(7)
driver.find_element(By.CSS_SELECTOR, 'div._aabd._aa8k._aanf').click()

time.sleep(5)

# 게시글 읽기
post_lst = []
for post in range(1,post_no+1):
    if post != 1:
        # 다음 페이지 넘기기 
        driver.find_element(By.CLASS_NAME, "_aaqg._aaqh").click()
        time.sleep(5)
        
    # 게시글 정보 가져오기 
    ## 작성자, 게시글
    writer = driver.find_element(By.CSS_SELECTOR, 'span._aap6._aap7._aap8').text
    article = driver.find_element(By.CSS_SELECTOR, 'span._aacl._aaco._aacu._aacx._aad7._aade').text

    ## 좋아요
    likes = driver.find_element(By.CSS_SELECTOR, 'a > div._aacl._aaco._aacw._aacx._aada._aade').text
    likes = re.sub('[^0-9.]','',likes)

    ## 날짜
    post_date = driver.find_element(By.CSS_SELECTOR,  'time._aaqe').get_attribute('datetime')
    post_date = post_date.split('.')[0].replace('T','')
    post_date = datetime.strptime(post_date,'%Y-%m-%d%H:%M:%S')
    # post_date = driver.find_element(By.CSS_SELECTOR,  'time._aaqe').text
    # post_date = datetime.strptime(post_date, '%m월 %d, %Y')

    post_lst.append([writer, article, likes, post_date])
    
    # 이미지 저장         
    i = 1
    print(f'{post}번째 포스트 이미지', end=' ')
    
    while True:
        try:
            if i == 1:
                img_url = driver.find_elements(By.CSS_SELECTOR, 'div._aagu._aato > div._aagv > img')[0].get_attribute('src')
            else:
                img_url = driver.find_elements(By.CSS_SELECTOR, 'div._aagu._aato > div._aagv > img')[1].get_attribute('src')
            img_reponse = requests.get(img_url)
            
            filename = 'article' + str(post) + '_img' + str(i) + '.png'
            with open(file_path + filename, 'wb') as f:
                f.write(img_reponse.content)
            
            print(f'{i}', end=' > ')
            i += 1
            
            driver.find_element(By.CSS_SELECTOR, 'button._afxw').click()
            time.sleep(0.5)

        except: 
            print('> 저장 완료!')
            break 
        
end = '크롤링 작업을 완료했습니다'
print('='*10,end.center(16),'='*10)


columns = ['작성자', '게시글', '좋아요', '작성일']
df_post = pd.DataFrame(post_lst, columns = columns)
df_post

===========  크롤링 작업을 시작합니다  ===========
1번째 포스트 이미지 1 > > 저장 완료!
2번째 포스트 이미지 1 > 2 > 3 > 4 > > 저장 완료!
3번째 포스트 이미지 1 > 2 > 3 > 4 > > 저장 완료!
==========  크롤링 작업을 완료했습니다  ==========


,작성자,게시글,좋아요,작성일
0,multicampus.official,올해 여러분과 함께라서 행복했습니다 😊🎉\n⠀\n가족들과 따뜻한 연말 보내시고\n2...,11,2021-12-27 01:23:48
1,multicampus.official,초·중·고등학생이라면 누구나 참여 가능한\nOPIc ROOKIE challenge ...,7,2021-10-05 06:07:56
2,multicampus.official,[10/8까지] 미라콤아이앤씨 협력회사 취업연계 IT 아카데미 교육생 모집\n⠀\n...,10,2021-09-27 01:43:33
